In [9]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
sys.path.append("../../")
sys.path.append("../../src/model")

%matplotlib inline
import wandb # This is optinal 
from src.utils.sequence import seqs_to_one_hot
wandb.login(host = "https://api.wandb.ai")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [10]:
# Load data from Another Project 

run = wandb.init(entity ='fderc_diffusion', project="Diffusion-DNA-RNA") # Change depending on your proejcts
artifact = run.use_artifact('DNA-dataset:v0')
dir = artifact.download()
wandb.finish()

wandb: Downloading large artifact DNA-dataset:v0, 3766.17MB. 5 files... 
wandb:   5 of 5 files downloaded.  
Done. 0:0:2.4


In [12]:
# Check Data 
datafile = pd.read_csv("artifacts/DNA-dataset:v0/dataset.csv.gz")
datafile.head()

,Unnamed: 0,chrom,seq,hepg2,k562,sknsh
0,0,chr7,CCTGGTCTTTCTTGCTAAATAAACATATCGTGCATCATCCAGATCT...,0.233601,0.060779,0.047194
1,1,chr1,CATAAAGATGAGGCTTGGCAAAGAACATCTCTCGGTGCCTCCCATT...,0.004565,0.379639,-0.244395
2,2,chr1,CATAAAGCCAATCACTGAGATGACAAGTACTGCCAGGAAAGAAGGC...,0.384537,0.036707,-0.004578
3,3,chr15,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...,4.116494,4.508784,3.040183
4,4,chr12,CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTT...,1.423444,1.616602,1.335892


In [13]:
# Select data 
small_data = datafile.loc[ (datafile['chrom'] =="chr1") | (datafile['chrom'] =="chr2") |(datafile['chrom'] =="chr3") |(datafile['chrom'] =="chr4")  ]

In [14]:
from src.utils.sequence import seqs_to_one_hot

seq_x = [seqs_to_one_hot(seq)[:, 0, :] for seq in small_data['seq']]
x = np.array(seq_x)
print(x.shape) # Data Size 

(203842, 200, 4)


In [15]:
y = np.array(small_data['hepg2'])
y1 = (np.quantile(y,0.95)<y) * 1 # Make a binary level 

In [8]:
run = wandb.init(entity ='fderc_diffusion', project="Diffusion-DNA-RNA", name = "make_data") # Change depending on your proejcts
run.log_artifact("./artifacts/DNA-dataset", name = 'DNA-dataset', type = "dataset")
wandb.finish()

wandb: Adding directory to artifact (./artifacts/DNA-dataset)... Done. 9.0s


In [19]:
''''
run = wandb.init(entity ='fderc_diffusion', project="Diffusion-DNA-RNA") # Change depending on your proejcts
run.log_artifact("artifacts/DNA-model", name="DNA-model", type="model")
wandb.finish()
'''

'\'\nrun = wandb.init(entity =\'fderc_diffusion\', project="Diffusion-DNA-RNA") # Change depending on your proejcts\nrun.log_artifact("artifacts/DNA-model", name="DNA-model", type="model")\nwandb.finish()\n'